<a href="https://colab.research.google.com/github/Caswell1083/Tjabadi-Caswell/blob/main/Day_3_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
!pip install pyspark


In [32]:
from pyspark.sql import SparkSession
import pandas as pd
spark = SparkSession.builder.appName('panda-and-spark').getOrCreate()

In [33]:
customer_raw = [(1, 'bob', 3462543658686),
           (2, 'rob', 9087567565439),
           (3, 'tim', 5436586999467),
           (4, 'tom', 8349756853250)]

customer_cols = ['id', 'name', 'credit_card_number']



orders_raw = [(1, 'ketchup', 'bob', 1.20),
           (2, 'rutabaga', 'bob', 3.35),
           (3, 'fake vegan meat', 'rob', 13.99),
           (4, 'cheesey poofs', 'tim', 3.99),
           (5, 'ice cream', 'tim', 4.95),
           (6, 'protein powder', 'tom', 49.95)]

orders_cols = ['id', 'product_name', 'customer', 'price']

In [34]:
customer_df = pd.DataFrame(customer_raw, columns=customer_cols)
orders_df = pd.DataFrame(orders_raw, columns=orders_cols)

customer_df

joined_df = pd.merge(customer_df, orders_df, how='inner', left_on='name', right_on='customer')
joined_df

## For self study. What happens if (4, 'tom', 8349756853250) in valuesA becomes (4, 'tod', 8349756853250)
## How do the results change?
## More sensibly; what if customers have not made any orders but we still require them in the result set?

,id_x,name,credit_card_number,id_y,product_name,customer,price
0,1,bob,3462543658686,1,ketchup,bob,1.20
1,1,bob,3462543658686,2,rutabaga,bob,3.35
2,2,rob,9087567565439,3,fake vegan meat,rob,13.99
3,3,tim,5436586999467,4,cheesey poofs,tim,3.99
4,3,tim,5436586999467,5,ice cream,tim,4.95
5,4,tom,8349756853250,6,protein powder,tom,49.95


In [35]:
customersDF = spark.createDataFrame(customer_raw, customer_cols)

ordersDF = spark.createDataFrame(orders_raw, orders_cols)

# Show tables
customersDF.show()
ordersDF.show()

+---+----+------------------+
| id|name|credit_card_number|
+---+----+------------------+
|  1| bob|     3462543658686|
|  2| rob|     9087567565439|
|  3| tim|     5436586999467|
|  4| tom|     8349756853250|
+---+----+------------------+

+---+---------------+--------+-----+
| id|   product_name|customer|price|
+---+---------------+--------+-----+
|  1|        ketchup|     bob|  1.2|
|  2|       rutabaga|     bob| 3.35|
|  3|fake vegan meat|     rob|13.99|
|  4|  cheesey poofs|     tim| 3.99|
|  5|      ice cream|     tim| 4.95|
|  6| protein powder|     tom|49.95|
+---+---------------+--------+-----+



In [36]:
joinedDF = customersDF.join(ordersDF, customersDF.name == ordersDF.customer)
joinedDF.show()

+---+----+------------------+---+---------------+--------+-----+
| id|name|credit_card_number| id|   product_name|customer|price|
+---+----+------------------+---+---------------+--------+-----+
|  1| bob|     3462543658686|  1|        ketchup|     bob|  1.2|
|  1| bob|     3462543658686|  2|       rutabaga|     bob| 3.35|
|  2| rob|     9087567565439|  3|fake vegan meat|     rob|13.99|
|  3| tim|     5436586999467|  4|  cheesey poofs|     tim| 3.99|
|  3| tim|     5436586999467|  5|      ice cream|     tim| 4.95|
|  4| tom|     8349756853250|  6| protein powder|     tom|49.95|
+---+----+------------------+---+---------------+--------+-----+



In [37]:
joined_df.groupby('name').agg({"price": ["sum"]})

,price
,sum
name,
bob,4.55
rob,13.99
tim,8.94
tom,49.95


In [38]:
import pyspark.sql.functions as f

joinedDF.groupby('name').agg(f.sum('price').alias('total')).show()

+----+-----------------+
|name|            total|
+----+-----------------+
| bob|             4.55|
| rob|            13.99|
| tim|8.940000000000001|
| tom|            49.95|
+----+-----------------+



In [39]:
# save to the filesystem to prevent another load
! curl -o rows.csv https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  427M    0  427M    0     0  3931k      0 --:--:--  0:01:51 --:--:-- 2744k


In [40]:
import pandas as pd
nyc_df = pd.read_csv('rows.csv')

<ipython-input-40-a70f860f02a9>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  nyc_df = pd.read_csv('rows.csv')


In [41]:
# number or rows

print(len(nyc_df))

# this is quite large so we will work with a sample while we experiment in pandas as least.


2118226


In [42]:
nyc_small = nyc_df.sample(frac=0.2, replace=False, random_state=1)

In [ ]:
# we are also going to limit the columns to those we are going to work with

nyc_small = nyc_small[['CRASH DATE', 'CONTRIBUTING FACTOR VEHICLE 1',
                       'BOROUGH', 'VEHICLE TYPE CODE 1',
                       'NUMBER OF PERSONS INJURED']]

In [43]:
nyc_small.head(2)

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
430841,05/19/2019,23:45,QUEENS,11368.0,40.753113,-73.860000,"(40.753113, -73.86)",108 STREET,37 DRIVE,NaN,...,Unspecified,NaN,NaN,NaN,4135557,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
1779911,04/06/2013,15:45,MANHATTAN,10018,40.754057,-73.995817,"(40.754057, -73.9958167)",WEST 35 STREET,9 AVENUE,NaN,...,Unspecified,NaN,NaN,NaN,28618,SPORT UTILITY / STATION WAGON,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN


In [44]:
from pyspark.sql import SQLContext


# there are nan's in the frame with strings, and spark can't 'infer the schema', so we have to help it out
# by replacing them with empty strings and forcing the column to be a string

sdf_small = SQLContext(spark).createDataFrame(nyc_small.fillna('').astype('str'))


# Lets check the schema quickly

print(sdf_small.schema)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


StructType([StructField('CRASH DATE', StringType(), True), StructField('CRASH TIME', StringType(), True), StructField('BOROUGH', StringType(), True), StructField('ZIP CODE', StringType(), True), StructField('LATITUDE', StringType(), True), StructField('LONGITUDE', StringType(), True), StructField('LOCATION', StringType(), True), StructField('ON STREET NAME', StringType(), True), StructField('CROSS STREET NAME', StringType(), True), StructField('OFF STREET NAME', StringType(), True), StructField('NUMBER OF PERSONS INJURED', StringType(), True), StructField('NUMBER OF PERSONS KILLED', StringType(), True), StructField('NUMBER OF PEDESTRIANS INJURED', StringType(), True), StructField('NUMBER OF PEDESTRIANS KILLED', StringType(), True), StructField('NUMBER OF CYCLIST INJURED', StringType(), True), StructField('NUMBER OF CYCLIST KILLED', StringType(), True), StructField('NUMBER OF MOTORIST INJURED', StringType(), True), StructField('NUMBER OF MOTORIST KILLED', StringType(), True), StructFiel

In [45]:
# Pandas
nyc_small.groupby('CRASH DATE')['CRASH DATE'].count().sort_values(ascending=False).head(5)

,CRASH DATE
CRASH DATE,
11/15/2018,224
01/21/2014,217
12/15/2017,210
02/03/2014,205
12/14/2018,203


In [46]:
nyc_small.groupby('BOROUGH')['BOROUGH'].count().sort_values(ascending=False).head(5)

,BOROUGH
BOROUGH,
BROOKLYN,92591
QUEENS,78223
MANHATTAN,65192
BRONX,43322
STATEN ISLAND,12247


In [47]:
nyc_small[(nyc_small.BOROUGH == 'QUEENS')]['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()

# you can also use a group by (to avoid the pandas value_counts function)

nyc_small[(nyc_small.BOROUGH == 'QUEENS')].groupby(
    'CONTRIBUTING FACTOR VEHICLE 1'
)['CONTRIBUTING FACTOR VEHICLE 1'].count().sort_values(ascending=False).head(5)

,CONTRIBUTING FACTOR VEHICLE 1
CONTRIBUTING FACTOR VEHICLE 1,
Unspecified,27318
Driver Inattention/Distraction,16548
Failure to Yield Right-of-Way,6763
Backing Unsafely,3878
Following Too Closely,2922


In [48]:
nyc_small.groupby(['VEHICLE TYPE CODE 1', 'BOROUGH'])['NUMBER OF PERSONS INJURED'].mean().sort_values(ascending=False).head(3)

,,NUMBER OF PERSONS INJURED
VEHICLE TYPE CODE 1,BOROUGH,
Limo,MANHATTAN,13.0
FRONT,BROOKLYN,12.0
PASSE,BRONX,11.0


In [49]:
# Check the column names
print(nyc_df.columns)




Index(['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME',
       'OFF STREET NAME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
       'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
       'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2',
       'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5'],
      dtype='object')


In [50]:
# Convert the 'CRASH DATE' column to datetime format, if needed
nyc_df['CRASH DATE'] = pd.to_datetime(nyc_df['CRASH DATE'], errors='coerce')

# Count the occurrences of each crash date and find the day with the most crashes
crash_by_day = nyc_df['CRASH DATE'].value_counts().idxmax()

print(f"The day with the most crashes: {crash_by_day}")


The day with the most crashes: 2014-01-21 00:00:00


In [51]:
# Count the number of crashes per borough
crash_by_location = nyc_df['BOROUGH'].value_counts().idxmax()

print(f"The place with the most crashes: {crash_by_location}")


The place with the most crashes: BROOKLYN


In [52]:
# Filter the dataframe for crashes in 'QUEENS'
queens_df = nyc_df[nyc_df['BOROUGH'] == 'QUEENS']

# Find the most common cause of accidents in Queens (using 'CONTRIBUTING FACTOR VEHICLE 1')
common_cause_in_queens = queens_df['CONTRIBUTING FACTOR VEHICLE 1'].value_counts().idxmax()

print(f"The most common cause of accidents in QUEENS: {common_cause_in_queens}")


The most common cause of accidents in QUEENS: Unspecified


In [53]:
# Group by 'VEHICLE TYPE CODE 1' and 'BOROUGH', then calculate the average number of injuries
average_injuries = nyc_df.groupby(['VEHICLE TYPE CODE 1', 'BOROUGH'])['NUMBER OF PERSONS INJURED'].mean().reset_index()

print(average_injuries)


     VEHICLE TYPE CODE 1    BOROUGH  NUMBER OF PERSONS INJURED
0                      0  MANHATTAN                        0.0
1                      1  MANHATTAN                        0.0
2                      1     QUEENS                        0.0
3                  11111   BROOKLYN                        0.0
4                  12 PA   BROOKLYN                        0.0
...                  ...        ...                        ...
2410          work truck  MANHATTAN                        3.0
2411               yello     QUEENS                        0.0
2412                  yy     QUEENS                        0.0
2413                omm  MANHATTAN                        0.0
2414                �MBU   BROOKLYN                        0.0

[2415 rows x 3 columns]
